# Test of readout code for LTC2324 ADC Chip

The overlay contains both the readout module for the ADC and a dummy module to generate simulated data.  The interface to the LTC2324 is through PMOD with the following pinout
- PMODA[0]: external trigger input, active HI
- PMODA[1]: cvt output, active LOW
- PMODA[2]: sck output
- PMODA[3]: clkout input
- PMODA[4]: sdo serial data input

The dummy ADC connections are on PMODB with pins corresponding to the same pins on PMODA
- PMODB[1]: dummy_cvt input
- PMODB[2]: dummy_sck input
- PMODB[3]: dummy_clkout output
- PMODB[4]: dummy_sdo serial data output

If PMODB pins 1-4 are jumpered to the same pins on on PMOD, then the module should read back the data written to the dummy_ADC register defined below.

In [1]:
# Because of a bug/feature of the Pynq library, this is necessary when loading updated
# versions of the same design.
from pynq import PL
PL.reset()

In [2]:
# load the overlay
from pynq import Overlay
pynq = Overlay("mani_readout.bit")
# help(pynq)

In [3]:
# Define the communication registers

# Interface to LTC2324 readout module
timing = pynq.timing   # timing control word
                       # timing[7:4] = number of clock cyccles for cvt low before first valid bit
                       # timing[3:0] = number of clock cycles for SCK_LO and SCK_HI
control = pynq.control # control word
                       # control[0] = arm
                       # control[1] = soft trigger
                       # control [6:2] - (not used)
                       # control [7] - polarity of external trigger 
                       #                 0-> active HI
                       #                 1-> active LO
state = pynq.state     # state of readount.  There will be data ready when state==4
# Data registers
data = [pynq.data1,pynq.data2,pynq.data3,pynq.data4]


# Interface to dummy data generation module
dummy_ADC = pynq.dummy_ADC  # Dummy data for the data simulation


# Utility routines

The following routines are used to arm and wait for triggers, as well as convert and display ADC values

In [4]:
# Arm the trigger
#    trigger_parity = 0 -> triggers on positive edge
#    trigger_parity = 1 -> triggers on negative edge
def arm(trigger_parity=1):
    parity_mask = trigger_parity<<7  # set bit 7 to parity
    # Toggle the lowest order bit to arm, while maintaining the parity mask
    control.write(0,1|parity_mask)
    control.write(0,parity_mask)

# Wait for a trigger by monitoring the state and looking for DONE (4)
DONE_state=4
def wait_for_DONE():
    while (state.read(0)!=DONE_state):  # poll the state
        pass
    return

# Convert a 16 bit 2s complement value to a signed python integer
# Courtesy of ChatGPT
def twos_complement_16bit(value):
    if value & (1 << 15):  # check if the sign bit is set
        return value - (1 << 16)
    else:
        return value

# Convert 16 bit, 2s complement ADC readout to voltage
Vref = 2.    # range from -Vref to +Vref
def voltage(raw):
    return(twos_complement_16bit(raw)*Vref/2**15)

# Print out the raw ADC values
def print_raw():
    print("Raw ADC values...")
    for i in range(4):
        print("  Channel %d = 0x%X"%(i,data[i].read(0)))


# Print out the voltages
def print_voltages():
    print("ADC input voltages...")
    for i in range(4):
        print("  Channel %d = %.4f V"%(i,voltage(data[i].read(0))))


# This routine creates four display fields for the voltages that can be updated

import ipywidgets as widgets
from ipywidgets import Text,VBox,Layout
from IPython.display import display

def create_voltage_display():
    # Create voltage displays as text boxes
    voltage_fields = [Text(value="(uninitialized)", description=f'Input {i}',layout=Layout(width="20%")) for i in range(4)]
    # put them into a box and display it
    display_box = VBox(voltage_fields)
    display(display_box)
    
    # Return the display fields
    return voltage_fields

# This routine will update the fields in "voltage_fiels" with the corrent input voltages
def update_voltage_display():
    for i in range(4):
        voltage_fields[i].value = "%.4f V"%voltage(data[i].read(0))

# Get one event and display it
def get_event():
    arm()                    # Arm the trigger
    wait_for_DONE()          # wait for DONE
    update_voltage_display()  # Display values

# Generic code follows...

In [5]:
voltage_fields=create_voltage_display()

In [ ]:
# Test loop
while True:
    get_event()
    print("Trigger received")
    

In [ ]:
# Print out the raw values
def print_raw():
    print("Raw ADC values...")
    for i in range(4):
        print("  Channel %d = 0x%X"%(i,data[i].read(0)))
